# Creating a numpy array of Stellar Kinematics from the Sim

This code is a copy of Luismi's code that reads in the simulation to produce a numpy array of the stellar kinematics. See more at: https://github.com/luismi98/astrophysics-research/blob/main/notebooks/other_notebooks/convert_sim_to_numpy.ipynb

# Libraries

In [14]:
import pynbody as pb
import pandas as pd
import numpy as np
import os
from galpy.util import coords as bovy_coords
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import os

# Constants

In [15]:
R0=8.1
Z0=0
pos_factor = 1.7
vel_factor=0.48
zabs = True
GSR = True

I_radius = 4
axisymmetric = False
angle = 27

STELLAR_MASS = 9.5*10**3  # stellar masses - see bottom left of page 8 in Debattista 2017

# Functions
The main function to create the numpy array

In [16]:


def load_and_save(simulation, save_path, angle=angle, axisymmetric=axisymmetric, pos_factor=pos_factor, vel_factor=vel_factor, R0=R0, Z0=Z0, zabs=zabs, GSR=GSR, I_radius=I_radius):

    sim = load_sim(simulation)
    print('sim loaded')

    if axisymmetric:
        df = convert_sim_to_df(sim.s, pos_factor=pos_factor, vel_factor=vel_factor, R0=R0, Z0=0, zabs=zabs, GSR=GSR, axisymmetric=axisymmetric)

        filename = build_filename(R0=R0,Z0=Z0,axisymmetric=True,zabs=zabs,pos_factor=pos_factor,GSR=GSR)

        save_as_np(df, save_path=save_path, filename='Luismi_data')

        return

    bar_angle = compute_bar_angle(sim, I_radius=I_radius)
    sim.rotate_z(-bar_angle)  # align with x axis

    sim.rotate_z(angle)  # anti-clockwise rotation as seen from above, as we will then flip y

    df = convert_sim_to_df(sim.s, pos_factor=pos_factor, vel_factor=vel_factor, R0=R0, angle=angle, zabs=zabs, GSR=GSR)
    print('df created')

    filename = build_filename(rot_angle=angle, R0=R0, Z0=Z0, axisymmetric=axisymmetric, zabs=zabs,
                              pos_factor=pos_factor, GSR=GSR)

    save_as_np(df, save_path=save_path, filename=filename)
    print('filed saved')
    


## Functions to Read in simulation and finds the bar angle
These functions read in the simulation and calculates the bar angle in order to rotate the simulation so that the galaxy is aligned along the bar of the Milky Way

In [17]:


# reads in the simulation
def load_sim(gz_file):

    sim = pb.load(gz_file)  # simulation from LuisMi (San Martin Fernandez et al. 2021)

    pb.analysis.halo.center(sim, mode="hyb")  # centre the simulation to the halo

    pb.analysis.angmom.faceon(sim, cen=(0, 0, 0))  # makes the simulation face on

    return sim


def compute_bar_angle(sim, I_radius=4):
    bar_stars = get_bar_stars(sim=sim, Rmax=I_radius)

    x_bar, y_bar, mass_bar = extract_xymass_from_stars(bar_stars)

    maj_axis_angle = calculate_bar_angle_from_inertia_tensor(x_bar, y_bar, mass_bar)
    print('bar angle calculated')

    return maj_axis_angle


def get_bar_stars(sim, tform_min=3, tform_max=6, zmin=0.2, Rmax=4):
    """
    - tform_min and tform_max default to 3 and 6, choosing stars between 4 and 7 years old
    - zmin defaults to 0.2 to avoid the spheroidal densest region in the plane
    """
    print('bar stars found')
    return sim.s[
            (sim.s['tform'] > tform_min)&
            (sim.s['tform'] < tform_max)&
            (np.abs(sim.s['z'].in_units("kpc")) > zmin)&
            (np.hypot(sim.s["x"].in_units("kpc"), sim.s["y"].in_units("kpc")) < Rmax)
        ]


def extract_xymass_from_stars(stars):
    x = np.array(stars['x'].in_units('kpc'))
    y = np.array(stars['y'].in_units('kpc'))
    mass = np.array(stars['mass'].in_units('Msol'))

    return x,y,mass


def calculate_bar_angle_from_inertia_tensor(x, y, mass):
    if not len(x) == len(y) == len(mass):
        raise ValueError("The length of the star's positions and mass arrays did not match.")

    # Calculate the moment of inertia tensor
    I_xx, I_yy, I_xy = np.sum(mass * y ** 2), np.sum(mass * x ** 2), -np.sum(mass * x * y)
    I = np.array([[I_xx, I_xy], [I_xy, I_yy]])

    # Calculate the eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(I)

    # The major axis (direction of the bar) is the direction of the eigenvector with smallest eigenvalue
    # That is because the bar has the smallest moment of inertia when rotated around the longitudinal axis of the bar
    index_lowest_eigenvalue = eigenvalues.argmin()
    major_axis = eigenvectors[:, index_lowest_eigenvalue]

    # Get the angle the major axis makes with the horizontal axis
    return np.degrees(np.arctan2(major_axis[1], major_axis[0]))



## Functions to read in velocity, pos of stars
These functions read in the position and velocity of the stars in the simulation.
The positions and velocities are then are manipulated by a position and velocity scale factor so they replicate the positions and velocities of stars in the Milky Way.
These values are then caluclated for different coordinate systems.

In [18]:


def convert_sim_to_df(sim_stars, pos_factor=1.7, vel_factor=0.48, R0=8.1, angle=27, zabs=True, GSR=True, axisymmetric=False):
    positions = np.array(sim_stars['pos'].in_units('kpc'))
    velocities = np.array(sim_stars['vel'].in_units('km s**-1'))
    tform = np.array(sim_stars['tform'])  # https://pynbody.github.io/pynbody/reference/derived.html

    df = pd.DataFrame()
    df['x'], df['y'], df['z'] = positions[:, 0], positions[:, 1], positions[:, 2]
    df['vx'], df['vy'], df['vz'] = velocities[:, 0], velocities[:, 1], velocities[:, 2]
    df['age'] = tform.max() - tform
    print('found stars pos, vel, age')

    if axisymmetric:
        axisymmetrise(df)
        angle = None

    MW_factor(df, pos_factor, vel_factor)
    MW_rot(df)


    # mirror below the plane onto above the plane to increase stats
    if zabs:
        df.loc[df.z < 0, 'vz'] *= -1
        df.loc[df.z < 0, 'z'] *= -1

    transform_coordinates(df, R0=R0, Z0=Z0, rot_angle=angle, GSR=GSR)
    print('calculate different coordinates')

    df.drop(columns=['X', 'Y', 'Z', 'vX', 'vY', 'vZ', 'pmlcosb', 'pmb'], inplace=True)
    return df


# changes pos, vel to match MW
def MW_factor(df, pos_f, vel_f):

    df.x = df.x*pos_f
    df.y = df.y*pos_f
    df.z = df.z*pos_f

    df.vx = df.vx*vel_f
    df.vy = df.vy*vel_f
    df.vz = df.vz*vel_f


# changes vel direction to match MW
def MW_rot(df):
    df.y = df.y * (-1)
    df.vy = df.vy * (-1)

    df.z = df.z * (-1)
    df.vz = df.vz * (-1)
    
    

## Functions for calculating coordinate systems

In [19]:


def transform_coordinates(df, R0=R0, Z0=Z0, GSR=True, rot_angle=None):

    v_sun = [12.9, 245.6, 7.78]
    
    if GSR:
        v_sun = [0,0,0]

    xyz_to_XYZ(df, R0=R0, Z0=Z0)
    print('x y z to X Y Z')
    vxvyvz_to_vXvYvZ(df, v_sun=v_sun, R0=R0, Z0=Z0)
    print('vx vy vz to vX vY vZ')

    XYZ_to_lbd(df)
    print('X Y Z to l d z')
    vXvYvZ_to_vrpmlpmb(df)
    print('vX vY vZ to vr pm lpmb')

    pmlpmb_to_vlvb(df)
    print('pml pmb to vl vb')

    lb_to_radec(df)
    print('l b to ra dec')
    pmlpmb_to_pmrapmdec(df)
    print('pml pmb to pmra pmdec')

    xyz_to_Rphiz(df)
    print('x y z to R phi z')
    vxvyvz_to_vRvphivz(df)
    print('vx vy vz to vR vphi vz')

    if rot_angle is not None:
        vxvy_to_vMvm(df, rot_angle=rot_angle)
        print('vx vy to vM vm')


def xyz_to_XYZ(df, R0=R0, Z0=Z0):
    XYZ=bovy_coords.galcenrect_to_XYZ(df.x.values,df.y.values,df.z.values, Xsun=-R0,Zsun=Z0)
    df['X'],df['Y'],df['Z'] = XYZ[:,0], XYZ[:,1], XYZ[:,2]


def vxvyvz_to_vXvYvZ(df,v_sun,R0=R0,Z0=Z0):
    # If v_sun is [0,0,0], the only thing this will do is perform a tiny rotation to align with astropy's frame definition
    vXYZ=bovy_coords.galcenrect_to_vxvyvz(df.vx.values,df.vy.values,df.vz.values,
                                        vsun=v_sun,Xsun=-R0,Zsun=Z0)
    df['vX'],df['vY'],df['vZ'] = vXYZ[:,0], vXYZ[:,1], vXYZ[:,2]


def XYZ_to_lbd(df):
    lbd=bovy_coords.XYZ_to_lbd(df.X.values,df.Y.values,df.Z.values,degree=True)
    df['l'],df['b'],df['d'] = lbd[:,0], lbd[:,1], lbd[:,2]

    # Set longitude range from 0,360 to -180,180
    df.loc[df.l>180, 'l'] -= 360


def vXvYvZ_to_vrpmlpmb(df):
    #vr in km/s, pmlcosb and pm (proper motion) in mas/yr (milli-arcsecond per year)
    vrpmlpmb=bovy_coords.vxvyvz_to_vrpmllpmbb(df.vX.values,df.vY.values,df.vZ.values,
                                        df.l.values,df.b.values,df.d.values,
                                        degree=True)
    df['vr'],df['pmlcosb'],df['pmb'] = vrpmlpmb[:,0], vrpmlpmb[:,1], vrpmlpmb[:,2]


def pmlpmb_to_vlvb(df):
    df['vl'] = convert_pm_to_velocity(df.d.values,df.pmlcosb.values)
    df['vb'] = convert_pm_to_velocity(df.d.values,df.pmb.values)


def convert_pm_to_velocity(distance, pm, kpc_bool=True, masyr_bool=True):

    velocity = distance * pm

    if kpc_bool:
        velocity *= get_conversion_kpc_to_km()

    if masyr_bool:
        velocity *= get_conversion_mas_to_rad() / get_conversion_yr_to_s()

    return velocity


def get_conversion_kpc_to_km():
    """
    1 kpc = 3.086e16 km
    """
    return 3.086e16


def get_conversion_mas_to_rad():
    """
    1 mas = 10^-3 * (1 deg / 3600 arcsec) * (pi rad / 180 deg) = 10^3*pi/(3600*180) rad
    """
    return 1e-3*np.pi/(180 * 3600)


def get_conversion_yr_to_s():
    """
    1yr = 1yr * 365 day/yr * 24 h/day * 3600 s/h = 3.1536e7 s
    """
    return 365*24*3600


def lb_to_radec(df):
    radec=bovy_coords.lb_to_radec(df.l.values,df.b.values,degree=True)
    df['ra'],df['dec']=radec[:,0],radec[:,1]


def pmlpmb_to_pmrapmdec(df):
    pmradec=bovy_coords.pmllpmbb_to_pmrapmdec(df.pmlcosb.values,df.pmb.values,
                                     df.l.values,df.b.values,degree=True)
    df['pmra'],df['pmdec']=pmradec[:,0],pmradec[:,1]


def xyz_to_Rphiz(df):

    o_Rphiz = bovy_coords.rect_to_cyl(df['x'], df['y'], df['z'])
    df['R'],df['phi'],df['z'] = o_Rphiz[0],np.degrees(o_Rphiz[1]),o_Rphiz[2]  #Note: converting phi to degrees

    #Phi between 0 and 360
    df.loc[df.phi < 0, 'phi'] += 360


def vxvyvz_to_vRvphivz(df):
    # Note strangely enough the cylindrical functions return tuples, unlike the others

    vRphiz = bovy_coords.rect_to_cyl_vec(df['vx'],df['vy'],df['vz'],df['x'],df['y'],df['z'])
    df['vR'],df['vphi'],df['vz'] = vRphiz[0],vRphiz[1],vRphiz[2]


def vxvy_to_vMvm(df,rot_angle=angle):
    """
    Get velocities along the bar axes.
    """

    c = np.cos(np.radians(rot_angle))
    s = np.sin(np.radians(rot_angle))

    df['vM'] = c*df['vx']-s*df['vy'] # semi-major axis
    df['vm'] = s*df['vx']+c*df['vy'] # semi-minor axis



## Functions to create numpy array

In [20]:


def build_filename(choice="708main",rot_angle=27,R0=R0,Z0=Z0,axisymmetric=False,zabs=True,pos_factor=1.7,GSR=True):
    bar_string = '' if axisymmetric else f'_bar{rot_angle}'
    scaling_string = "_scale" + str(pos_factor)
    frame_string = '' if GSR else '_LSR'
    zabs_string = '' if zabs else '_noZabs'
    axi_string = '_axisymmetric' if axisymmetric else ''
    R0_string = f'_{R0}R0'
    Z0_string = f"_{Z0}Z0" if Z0 != Z0 else ''
    print('file name created')

    return f"{choice}_MWout{bar_string}{scaling_string}{R0_string}{Z0_string}{frame_string}{zabs_string}{axi_string}_test"


def save_as_np(df, save_path, filename):

    sim_array = np.array(df.values.astype(np.float32))
    column_array = np.array(df.columns.to_list())

    np.save(save_path + filename, sim_array)
    print("Saved:", save_path + filename)


    np.save(save_path + filename + "_columns", column_array)
    print("Saved:", save_path + filename + "_columns")

    with open(save_path + filename + "_columns.txt", 'w') as f:
        f.write(f"Saved simulation with datatype np.float32.\nThe columns are: {list(column_array)}")
        
        

In [21]:
load_and_save('../../run708main.01000', '')

/Users/benmcdonald/opt/anaconda3/lib/python3.9/site-packages/pynbody/snapshot/tipsy.py:124: RuntimeWarning: No readable param file in the run directory or parent directory: using defaults.
  warnings.warn(

sim loaded
bar stars found
bar angle calculated
found stars pos, vel, age
x y z to X Y Z
vx vy vz to vX vY vZ
X Y Z to l d z
vX vY vZ to vr pm lpmb
pml pmb to vl vb
l b to ra dec
pml pmb to pmra pmdec
x y z to R phi z
vx vy vz to vR vphi vz
vx vy to vM vm
calculate different coordinates
df created
file name created
Saved: 708main_MWout_bar27_scale1.7_8.1R0_test
Saved: 708main_MWout_bar27_scale1.7_8.1R0_test_columns
filed saved
